In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
from sklearn.metrics import normalized_mutual_info_score
from huggingface_hub import login

import asyncio
import tensorflow as tf
import json
from hugchat import hugchat

load_dotenv()
LLAMA = os.getenv("LLAMA")
login(token=LLAMA)

#get the cookies
base_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
json_file_path = os.path.join(base_path, 'cookies.json')
with open(json_file_path, 'r') as file:
    cookies = json.load(file)


# text is regarded as key
seven_newsgroups_data = pd.read_csv('fetch_7newsgroups.csv')

def split_train_test(data):
    # Remove Nan
    data = data.dropna(how="any")
    train, test = train_test_split(data, test_size=0.05, random_state=42)

    train = train.sort_values(by='label')
    train_first_doc = train.groupby('label').first().reset_index()

    train_first_doc['div'] = 'train'
    test['div'] = 'test'

    return test, train_first_doc

test_df, instruction_df = split_train_test(seven_newsgroups_data)
test_df["name"] = range(1, len(test_df) + 1)
print(test_df.iloc[5])
print("test Length: ",len(test_df))
print("instruction Length: ", len(instruction_df))
print("original length: ", len(seven_newsgroups_data))


C:\Users\super161\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unnamed: 0                                                    5545
text             From: sp1henhj@edit (Henrik Balthazar Hjort)\n...
label20                                                          5
labels_name20                                       comp.windows.x
label                                                   Technology
div                                                           test
name                                                             6
Name: 5545, dtype: object
test Length:  893
instruction Length:  7
original length:  18846


In [2]:
def make_prompts(bbc_instructions, bbc_data):
    prompts = []
    
    # General instructions and fixed texts
    general_instruction = (
        "You are a perfect topic modeling machine. Given a text and the different topics, "
        "you will classify the texts to the correct topic. First you will receive the topics, "
        "afterwards an example and finally the text you have to assign one of the before mentioned topics to."
    )
    topics = "The topics are sport, technology, politics, vehicles, religion, miscellaneous and science. Please make sure, you know the topics and their meaning."
    transition_to_examples = "Now an example for each of the categories will follow."
    transition_to_text_to_classify = (
        "Now the text, you have to classify will follow. Please assess its topic and answer only the topic of it."
    )

    # Iterate through the test DataFrame rows
    for _, test_row in bbc_data.iterrows():
        prompt = general_instruction + "\n" + topics + "\n" + transition_to_examples + "\n"

        # Iterate through instruction DataFrame to add examples
        for _, instruction_row in bbc_instructions.iterrows():
            category = instruction_row['label']
            example_text = instruction_row['text']
            prompt += f"For the following text: \n{example_text}\nThe correct answer would be: {category}\n"

        # Add the actual text to classify from the test set
        text_to_classify = test_row['text']
        prompt += transition_to_text_to_classify + "\n" + text_to_classify + "\n"
        name = f"{test_row['name']}"
        #print(name)
        prompt_dict = {}
        prompt_dict[name] = prompt
        prompts.append(prompt_dict)
    return prompts            

prompts = make_prompts(instruction_df, test_df)
print(len(prompts))

893


In [3]:
async def huggingchat(prompt):
    semaphore = asyncio.Semaphore(5)
      
    async with semaphore:
        chatbot = hugchat.ChatBot(cookies)  
        name = list(prompt.keys())[0]
        promptAI = prompt[name]
        # Create a new conversation
        id = chatbot.new_conversation()
        chatbot.change_conversation(id)

        #print(chatbot.chat(promptAI))
        answer_from_chatbot = chatbot.chat(promptAI)
        try:
            answer: str = answer_from_chatbot['text']
        except:
            answer = ""
        return {name: answer}

In [4]:

async def prompt_orchestrator(prompts):
    batch_size = 5
    avatiables = []
    batches = [prompts[i:i + batch_size] for i in range(0, len(prompts), batch_size)]

    
    if len(batches[-1]) < batch_size:
        remaining = len(batches[-1])
        batches[-1] = prompts[-remaining:]

    
    if len(prompts) % batch_size != 0:
        remaining = len(prompts) % batch_size
        batches[-1] = prompts[-remaining:]

    max_retries = 5  
    retry_count = 0  
    current_batch_index = 0  

    while retry_count <= max_retries:
        try:
            counter = current_batch_index * batch_size  
            for batch_index in range(current_batch_index, len(batches)):
                batch = batches[batch_index]
                avatiables_batch = []

                for prompt in batch:
                    response = await huggingchat(prompt)
                    avatiables_batch.append(response)
                    await asyncio.sleep(6)  

                avatiables.extend(avatiables_batch)
                counter += batch_size
                await asyncio.sleep(8)  
                print(f"{counter} prompts processed.")
                current_batch_index = batch_index + 1  
                retry_count = 0  

            break  

        except Exception as e:
            retry_count += 1
            if retry_count > max_retries:
                print(f"Failed after {max_retries} retries. Returning the accumulated results.")
                return avatiables
            print(f"An error occurred: {e}. Retrying in 15 seconds... (Attempt {retry_count}/{max_retries})")
            await asyncio.sleep(15)  

    return avatiables

results = await prompt_orchestrator(prompts)

print(results)

An error occurred: Extra data: line 2 column 1 (char 11921). Retrying in 15 seconds... (Attempt 1/5)


CancelledError: 

In [6]:

############
 
# load model and tokenizer
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

async def prompt_llama(prompt):
    semaphore = asyncio.Semaphore(5)
    async with semaphore:

        name = list(prompt.keys())[0]
        promptAI = prompt[name]
        print(f"Prompt:", promptAI)
        #tokenize prompt
        inputs = tokenizer(promptAI, return_tensors="pt")

         # Generate text in a thread pool
        generate_ids = await asyncio.to_thread(
            model.generate, 
            inputs['input_ids'], 
            attention_mask=inputs.get('attention_mask'),
            max_new_tokens = 30,
            pad_token_id=tokenizer.eos_token_id 
        )
        #Decode 
        text_response = tokenizer.batch_decode(
            generate_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )[0]

        print(f"Response:", text_response)
        return {name: text_response}
        

In [10]:
##########
async def prompt_orchestrator():
      batch_size = 5
      avatiables = []
      batches = [prompts[i:i + batch_size] for i in range(0, len(prompts), batch_size)]

      # Ensure the last batch is smaller if it's less than batch_size
      if len(batches[-1]) < batch_size:
            remaining = len(batches[-1])
            batches[-1] = prompts[-remaining:]

      # If the last batch is part of the earlier slices and it's less than batch_size
      if len(prompts) % batch_size != 0:
            remaining = len(prompts) % batch_size
            batches[-1] = prompts[-remaining:]

      try:
            for batch in batches:
                  avatiables_batch = await asyncio.gather(*(prompt_llama(prompt) for prompt in batch))
                  avatiables.extend(avatiables_batch)
                  print("sleeping")
                  await asyncio.sleep(1)

      except Exception as e:
            print(f"An error occurred: {e}. Returning the accumulated results.")
            return avatiables
      
      return avatiables

results = await prompt_orchestrator()

print(results)

Prompt: You are a perfect topic modeling machine. Given a text and the different topics, you will classify the texts to the correct topic. First you will receive the topics, afterwards an example and finally the text you have to assign one of the before mentioned topics to.
The topics are sport, technology, politics, vehicles, religion, miscellaneous and science. Please make sure, you know the topics and their meaning.
Now an example for each of the categories will follow.
For the following text: 
From: 02106@ravel.udel.edu (Samuel Ross)
Subject: Desert storm cards for sale!!!
Distribution: usa
Organization: University of Delaware
Lines: 23
Nntp-Posting-Host: ravel.udel.edu

I have some Desert Storm cards for sale.  I have divided them up into two lots.  I will take the highest bid before 4/30/93.  Just drop me a line with your bid.


Lot #1 Desert Storm (Coalition for Peace, Topps)

 cards# 38,3,16,28(S),8(S),71,30,34*2,69,48,2,79,15,16(S),66,69,17,21,26
        85,25,17(S),18,6,86,23

In [11]:
#########################################################

from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
print("initialized ")
prompt = "Tell me a joke about clowns."
inputs = tokenizer(prompt, return_tensors="pt")
print("tokenized ")
# Generate
generate_ids = model.generate(
    inputs['input_ids'], 
    attention_mask=inputs.get('attention_mask'), 
    max_length=30
)

print("generated ")
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


initialized 
tokenized 
generated 


'Tell me a joke about clowns. A clown has a big belly. He gets up and goes to the bathroom. He looks in the mirror'

In [5]:
def add_correct_anser(results, test_df):
    result_with_correct_answer = []

    for result in results:
        for key, value in result.items():
            value = value.replace("\n", "")
            value = value.replace(" \n", "")
            value = value.replace(" ","")
            value = value.replace("*","")
            value = value.lower()
            matching_rows = test_df.loc[test_df['name'] == int(key), 'label']
            if not matching_rows.empty:
                if type(matching_rows.values[0]) == float: 
                    print(f"Nan value", matching_rows.values[0])
                else:
                    category_value = matching_rows.values[0]
                    category_value = category_value.lower()
                    result_with_correct_answer.append({key: (value, category_value)})

            else:
                print(f"No matching category found for key: {key}")
                result_with_correct_answer.append({key: (value, None)})
            

    return result_with_correct_answer
results_with_correct_answer = add_correct_anser(results, test_df)
print(len(results_with_correct_answer))
print(results_with_correct_answer)
print("to forward",len(results))

893
[{'1': ('miscellaneous', 'vehicles')}, {'2': ('religion', 'religion')}, {'3': ('politics', 'science')}, {'4': ('miscellaneous', 'religion')}, {'5': ('thetopicofthistextis:technology', 'technology')}, {'6': ('science', 'science')}, {'7': ('science', 'technology')}, {'8': ('vehicles', 'miscellaneous')}, {'9': ('thecorrectanswerwouldbe:politics', 'politics')}, {'10': ('technology', 'technology')}, {'11': ('thetopicofthistextis:sport', 'sport')}, {'12': ('politics', 'religion')}, {'13': ('technology', 'religion')}, {'14': ('technology', 'science')}, {'15': ('thecorrectanswerwouldbe:technology', 'science')}, {'16': ('technology', 'technology')}, {'17': ('vehicles', 'vehicles')}, {'18': ('politics', 'science')}, {'19': ('miscellaneous', 'science')}, {'20': ('science', 'politics')}, {'21': ('thecorrectansweris:vehicles', 'miscellaneous')}, {'22': ('politics', 'politics')}, {'23': ('politics', 'politics')}, {'24': ('science', 'science')}, {'25': ('thecorrectanswerwouldbe:miscellaneous', 's

In [9]:
import difflib
from thefuzz import fuzz
def post_process(results):
      test_df.loc['result'] = None
      string_list = ["religion", "politics","sport","technology", "vehicles", "science","miscellaneous"]
      final_results = []
      for result in results:
            similarity = []
            for string in string_list:
                  similarity.append([fuzz.partial_ratio(list(result.values())[0][0], string),string])
            max_similarity = max(similarity)
            
            if (max_similarity[0]>60):
                  final_results.append({list(result.keys())[0]: (max_similarity[1], list(result.values())[0][1])})
                  test_df.loc[test_df['name'] == list(result.keys())[0], 'result'] = max_similarity[1]
            else:
                  final_results.append({list(result.keys())[0]: ("", list(result.values())[0][1])})
                  test_df.loc[test_df['name'] == list(result.keys())[0], 'result'] = ""
      
      test_df.to_csv(r"E:\uni\NLP group project\llama_7newsgroups.csv", index=False)
      return final_results
result_with_correct_answer_pre_post_process = results_with_correct_answer
print(result_with_correct_answer_pre_post_process)
results_with_correct_answer = post_process(results_with_correct_answer)
print(results_with_correct_answer)

[{'1': ('miscellaneous', 'vehicles')}, {'2': ('religion', 'religion')}, {'3': ('politics', 'science')}, {'4': ('miscellaneous', 'religion')}, {'5': ('thetopicofthistextis:technology', 'technology')}, {'6': ('science', 'science')}, {'7': ('science', 'technology')}, {'8': ('vehicles', 'miscellaneous')}, {'9': ('thecorrectanswerwouldbe:politics', 'politics')}, {'10': ('technology', 'technology')}, {'11': ('thetopicofthistextis:sport', 'sport')}, {'12': ('politics', 'religion')}, {'13': ('technology', 'religion')}, {'14': ('technology', 'science')}, {'15': ('thecorrectanswerwouldbe:technology', 'science')}, {'16': ('technology', 'technology')}, {'17': ('vehicles', 'vehicles')}, {'18': ('politics', 'science')}, {'19': ('miscellaneous', 'science')}, {'20': ('science', 'politics')}, {'21': ('thecorrectansweris:vehicles', 'miscellaneous')}, {'22': ('politics', 'politics')}, {'23': ('politics', 'politics')}, {'24': ('science', 'science')}, {'25': ('thecorrectanswerwouldbe:miscellaneous', 'scien

In [10]:
def extract_ground_truth_and_predictions(results_with_correct_answer):
    ground_truth = []
    predictions = []
    for result in results_with_correct_answer:
        for key, value in result.items():
            ground_truth.append(value[1])
            predictions.append(value[0])
    return ground_truth, predictions
ground_truth, predictions = extract_ground_truth_and_predictions(results_with_correct_answer)


In [11]:
def calculate_NMI(ground_truth, predictions):
    
    nmi_score = normalized_mutual_info_score(ground_truth, predictions)
    print(f"Normalized Mutual Information Score: {nmi_score}")
    return nmi_score


In [12]:
import numpy as np
from collections import Counter

def calculate_purity(predicted_labels, true_labels):
    # Convert lists to numpy arrays for easier indexing
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)
    
    # Get unique clusters
    unique_clusters = np.unique(predicted_labels)
    
    # Total number of instances
    total_instances = len(true_labels)
    
    # Calculate the number of correctly classified instances in each cluster
    correctly_classified = 0
    for cluster in unique_clusters:
        # Get the true labels of instances in the current cluster
        indices_in_cluster = np.where(predicted_labels == cluster)[0]
        labels_in_cluster = true_labels[indices_in_cluster]
        
        # Determine the most common true label in this cluster
        majority_label_count = Counter(labels_in_cluster).most_common(1)[0][1]
        
        # Add the number of correctly classified instances in this cluster
        correctly_classified += majority_label_count
    
    # Calculate purity
    purity = correctly_classified / total_instances
    print(f"Purity: {purity}")
    return purity





In [13]:
def calculate_accuracy(predicted_labels, true_labels):
    # Ensure that the predicted_labels and true_labels have the same length
    if len(predicted_labels) != len(true_labels):
        raise ValueError("The length of predicted and true labels must be the same.")
    
    # Count the number of correct predictions
    correct_predictions = sum(1 for pred, true in zip(predicted_labels, true_labels) if pred == true)
    
    # Calculate accuracy
    accuracy = correct_predictions / len(true_labels)
    print(f"Accuracy: {accuracy}")
    return accuracy





In [14]:
from sklearn.metrics import f1_score
def calculate_f1_score(ground_truth, predictions):

    f1 = f1_score(ground_truth, predictions, average='micro') # Because there might be over/ under representation of some classes
    print(f"F1 Score: {f1}")
    return f1



In [15]:
calculate_NMI(ground_truth, predictions)
calculate_purity(predictions, ground_truth)
calculate_accuracy(predictions, ground_truth)
calculate_f1_score(ground_truth, predictions)

Normalized Mutual Information Score: 0.5015054134451503
Purity: 0.7066069428891377
Accuracy: 0.6786114221724524
F1 Score: 0.6786114221724524


0.6786114221724524